I plan to update the model in a timely manner. 

For the current situation, I referred to the following.Needs suppression as it is likely to be overfitted.¶

https://www.kaggle.com/code/swimmy/lgbm-opt-model-jpx

* This is probably the maximum in LB, as some are already in LB with a score of 5.434. 
* Therefore, from here on, it is necessary to suppress overfitting and increase the Private Score, rather than aiming at the value of LB.
https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/discussion/318015

# **if it is useful for you, please vote!**

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:

#prices["Date"]
prices

In [ ]:
NDAYS = 34
lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)


In [ ]:
lastdays

In [ ]:
lastdays2 = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]]
lastdays2

In [ ]:
lastdays

In [ ]:
lastdays = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
lastdays

In [ ]:
prices.Date = pd.to_datetime(prices.Date)
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
X=prices[["Date","SecuritiesCode","Avg","High","Open","Close","Low","Volume"]]
y=prices[["Target"]]
codes = X.SecuritiesCode.unique()

In [ ]:
y

In [ ]:
import optuna

def objectives(trial):
    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 200, 3000),
            'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
            'max_bin': trial.suggest_int('max_bin', 2, 100),
            'learning_rate': trial.suggest_uniform('learning_rate',0, 1),
    }

    model = LGBMRegressor(**params)
    model.fit(X,y)
    score = model.score(X,y)
    return score

In [ ]:
# opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
# opt.optimize(objectives, n_trials=1)

# # 最適パラメータ取得
# trial = opt.best_trial
# params_best = dict(trial.params.items())
# params_best['random_seed'] = 16

# # 最適パラメータで学習/予測    
# model_o = LGBMRegressor(**params_best)#


In [ ]:
model_o = LGBMRegressor(learning_rate=0.6818202991034834, max_bin=95, n_estimators=655,
              num_leaves=1263, random_seed=0)
model_o.fit(X,y)

In [ ]:
# model_o.fit(X,y)

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    sample_prediction["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
    df = sample_prediction[["Date","SecuritiesCode","Avg"]]
    df["High"] = prices["High"]
    df["Open"] = prices["Open"]
    df["Close"] = prices["Close"]
    df["Low"] = prices["Low"]
    df["Volume"] = prices["Volume"]
    df.Date = pd.to_datetime(df.Date)
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    print('df=',df)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)

In [ ]:
# for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
#     ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
#     print('ds=',ds)
#     sample_prediction["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
#     df = sample_prediction[["Date","SecuritiesCode","Avg"]]
#     df = sample_prediction[["Date","SecuritiesCode","Avg"]]
#     df["High"] = prices["High"]
#     df["Open"] = prices["Open"]
#     df["Close"] = prices["Close"]
#     df["Low"] = prices["Low"]
#     df["Volume"] = prices["Volume"]
#     df.Date = pd.to_datetime(df.Date)
#     df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)   
#     sample_prediction["Prediction"] = model_o.predict(df)
#     print('df=',df)
#     print('sample_prediction=',sample_prediction)